# Classification of Online Retail Customers Using Neural Networks

## CSE 5632 - Neural Networks Course Project

This notebook implements customer classification using RFM (Recency, Frequency, Monetary) analysis and compares multiple machine learning models:
- Neural Network (Multi-Layer Perceptron)
- Gradient Boosting Classifier
- Random Forest Classifier

**Dataset:** UCI Online Retail Dataset

## 1. Import Libraries

In [ ]:
# Data Manipulation & Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

# Preprocessing & Clustering
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.cluster import KMeans

# Model Selection & Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, accuracy_score, confusion_matrix,
    roc_curve, auc, f1_score, precision_score, recall_score
)

# Classification Models
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Settings
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load the Online Retail Dataset
# Note: Update the file path if running in a different environment
file_path = 'Online Retail.xlsx'

df = pd.read_excel(file_path)
print(f"Original Data Shape: {df.shape}")
df.head()

## 3. Data Preprocessing

In [ ]:
# Standardize column names
df.rename(columns={
    'InvoiceNo': 'Invoice',
    'CustomerID': 'Customer ID',
    'UnitPrice': 'Price'
}, inplace=True)

# Drop rows with null Customer ID
df = df.dropna(subset=['Customer ID'])

# Remove duplicate entries
df = df.drop_duplicates()

# Remove cancelled orders (invoices starting with 'C')
df['Invoice'] = df['Invoice'].astype(str)
df = df[~df['Invoice'].str.startswith('C')]

print(f"Data Shape after cleaning: {df.shape}")
print(f"Number of unique customers: {df['Customer ID'].nunique()}")
df.head()

## 4. RFM Feature Engineering

In [ ]:
# Calculate total transaction value
df['TotalSum'] = df['Quantity'] * df['Price']

# Set snapshot date for Recency calculation
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
print(f"Snapshot Date: {snapshot_date}")

# Aggregate data by Customer ID to compute RFM metrics
rfm = df.groupby(['Customer ID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'Invoice': 'nunique',                                      # Frequency
    'TotalSum': 'sum'                                          # Monetary
})

# Rename columns
rfm.rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'TotalSum': 'Monetary'
}, inplace=True)

# Filter to ensure positive monetary values
rfm = rfm[rfm['Monetary'] > 0]

print(f"RFM Table Shape: {rfm.shape}")
rfm.head()

In [ ]:
# Apply Log Transformation and Standard Scaling
rfm_log = np.log(rfm + 1)

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)

rfm_scaled_df = pd.DataFrame(rfm_scaled, index=rfm.index, columns=rfm.columns)
print("Data Preprocessed (Log + Scaled) successfully!")
rfm_scaled_df.head()

## 5. Customer Segmentation using K-Means Clustering

In [ ]:
# Apply K-Means Clustering with 3 clusters (Low, Mid, High value)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(rfm_scaled_df)

rfm['Cluster'] = kmeans.labels_

# Order clusters by monetary value (0=Low, 1=Mid, 2=High)
cluster_mapping = rfm.groupby('Cluster')['Monetary'].mean().sort_values().index
mapping_dict = {old_label: new_label for new_label, old_label in enumerate(cluster_mapping)}
rfm['Cluster'] = rfm['Cluster'].map(mapping_dict)

# Display cluster summary
cluster_summary = rfm.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Cluster': 'count'
}).rename(columns={'Cluster': 'Count'})

print("Cluster Summary (0=Low, 1=Mid, 2=High):")
display(cluster_summary.round(2))

rfm_scaled_df['Cluster'] = rfm['Cluster']

## 6. 3D Visualization of Customer Segments

In [ ]:
import plotly.express as px

rfm_vis = rfm.copy()
rfm_vis['Cluster_Label'] = rfm['Cluster'].map({0: 'Low', 1: 'Mid', 2: 'High'})

fig = px.scatter_3d(
    rfm_vis, x='Recency', y='Frequency', z='Monetary',
    color='Cluster_Label',
    title='3D View of Customer Segments',
    opacity=0.7,
    color_discrete_map={'Low': 'red', 'Mid': 'blue', 'High': 'green'}
)
fig.show()

## 7. Prepare Data for Classification

In [ ]:
# Define Features (X) and Target (y)
X = rfm_scaled_df.drop('Cluster', axis=1)
y = rfm_scaled_df['Cluster']

# Split the data (80% Train, 20% Test) with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training Data: {X_train.shape}")
print(f"Testing Data: {X_test.shape}")
print(f"\nClass Distribution in Training Set:")
print(y_train.value_counts().sort_index())

## 8. Model Training

### 8.1 Neural Network (Multi-Layer Perceptron)

In [ ]:
# Prepare target for Neural Network (One-Hot Encoding)
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Build the Neural Network Architecture
model_nn = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the Model
model_nn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
print("Training Neural Network...")
history = model_nn.fit(
    X_train, y_train_cat,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Get predictions
y_pred_nn_prob = model_nn.predict(X_test)
y_pred_nn = np.argmax(y_pred_nn_prob, axis=1)

nn_accuracy = accuracy_score(y_test, y_pred_nn)
print(f"\nNeural Network Test Accuracy: {nn_accuracy*100:.2f}%")

### 8.2 Gradient Boosting Classifier

In [ ]:
# Train Gradient Boosting Classifier
print("Training Gradient Boosting Classifier...")
model_gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
model_gb.fit(X_train, y_train)

# Get predictions
y_pred_gb = model_gb.predict(X_test)
y_pred_gb_prob = model_gb.predict_proba(X_test)

gb_accuracy = accuracy_score(y_test, y_pred_gb)
print(f"Gradient Boosting Test Accuracy: {gb_accuracy*100:.2f}%")

### 8.3 Random Forest Classifier

In [ ]:
# Train Random Forest Classifier
print("Training Random Forest Classifier...")
model_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
model_rf.fit(X_train, y_train)

# Get predictions
y_pred_rf = model_rf.predict(X_test)
y_pred_rf_prob = model_rf.predict_proba(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Test Accuracy: {rf_accuracy*100:.2f}%")

## 9. Model Evaluation

### 9.1 Classification Reports

In [ ]:
# Classification Report for all models
class_names = ['Low Value', 'Mid Value', 'High Value']

print("=" * 60)
print("NEURAL NETWORK - Classification Report")
print("=" * 60)
print(classification_report(y_test, y_pred_nn, target_names=class_names))

print("=" * 60)
print("GRADIENT BOOSTING - Classification Report")
print("=" * 60)
print(classification_report(y_test, y_pred_gb, target_names=class_names))

print("=" * 60)
print("RANDOM FOREST - Classification Report")
print("=" * 60)
print(classification_report(y_test, y_pred_rf, target_names=class_names))

### 9.2 Confusion Matrices

In [ ]:
# Create Confusion Matrices for all models
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models = [
    ('Neural Network', y_pred_nn),
    ('Gradient Boosting', y_pred_gb),
    ('Random Forest', y_pred_rf)
]

for ax, (name, y_pred) in zip(axes, models):
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues', ax=ax,
        xticklabels=class_names, yticklabels=class_names
    )
    ax.set_title(f'{name}\nAccuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('confusion_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

### 9.3 ROC Curves and AUC Scores

In [ ]:
# Binarize the labels for multi-class ROC
n_classes = 3
y_test_bin = label_binarize(y_test, classes=[0, 1, 2])

# Calculate ROC curves for each model and class
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

models_prob = [
    ('Neural Network', y_pred_nn_prob),
    ('Gradient Boosting', y_pred_gb_prob),
    ('Random Forest', y_pred_rf_prob)
]

colors = ['red', 'blue', 'green']

for ax, (model_name, y_prob) in zip(axes, models_prob):
    auc_scores = []
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_prob[:, i])
        roc_auc = auc(fpr, tpr)
        auc_scores.append(roc_auc)
        ax.plot(fpr, tpr, color=colors[i], lw=2,
                label=f'{class_names[i]} (AUC = {roc_auc:.3f})')

    ax.plot([0, 1], [0, 1], 'k--', lw=2)
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f'{model_name}\nMean AUC: {np.mean(auc_scores):.3f}')
    ax.legend(loc='lower right')

plt.tight_layout()
plt.savefig('roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

### 9.4 Model Performance Comparison

In [ ]:
# Calculate metrics for each model
def calculate_metrics(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'F1 (Macro)': f1_score(y_true, y_pred, average='macro'),
        'F1 (Weighted)': f1_score(y_true, y_pred, average='weighted'),
        'Precision (Macro)': precision_score(y_true, y_pred, average='macro'),
        'Recall (Macro)': recall_score(y_true, y_pred, average='macro')
    }

metrics_nn = calculate_metrics(y_test, y_pred_nn)
metrics_gb = calculate_metrics(y_test, y_pred_gb)
metrics_rf = calculate_metrics(y_test, y_pred_rf)

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Neural Network': metrics_nn,
    'Gradient Boosting': metrics_gb,
    'Random Forest': metrics_rf
}).T

print("Model Performance Comparison:")
display(comparison_df.round(4))

In [ ]:
# Visualize Model Comparison
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df.columns))
width = 0.25

models_list = ['Neural Network', 'Gradient Boosting', 'Random Forest']
colors = ['#2ecc71', '#3498db', '#e74c3c']

for i, model in enumerate(models_list):
    values = comparison_df.loc[model].values * 100
    bars = ax.bar(x + i*width, values, width, label=model, color=colors[i])
    ax.bar_label(bars, fmt='%.1f%%', fontsize=8)

ax.set_ylabel('Score (%)')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x + width)
ax.set_xticklabels(comparison_df.columns, rotation=15, ha='right')
ax.legend()
ax.set_ylim([0, 110])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Training History (Neural Network)

In [ ]:
# Plot Training History
plt.figure(figsize=(12, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Neural Network - Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(alpha=0.3)

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Neural Network - Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('nn_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

## 11. Summary

This project successfully demonstrated the application of neural networks and machine learning classifiers for customer segmentation based on RFM analysis.

**Key Findings:**
- All three models achieved high accuracy on the customer classification task
- The Neural Network model demonstrated excellent performance with minimal overfitting
- RFM features proved to be highly effective for customer value segmentation

**Customer Segments Identified:**
- **Low Value (Cluster 0):** At-risk/churned customers with high recency and low engagement
- **Mid Value (Cluster 1):** Regular customers with moderate purchasing behavior
- **High Value (Cluster 2):** VIP customers with recent purchases and high monetary value

In [ ]:
# Final Summary Table
print("=" * 70)
print("FINAL MODEL PERFORMANCE SUMMARY")
print("=" * 70)

summary_data = {
    'Model': ['Neural Network', 'Gradient Boosting', 'Random Forest'],
    'Accuracy': [f'{nn_accuracy*100:.2f}%', f'{gb_accuracy*100:.2f}%', f'{rf_accuracy*100:.2f}%'],
    'F1 Score (Macro)': [
        f"{metrics_nn['F1 (Macro)']*100:.2f}%",
        f"{metrics_gb['F1 (Macro)']*100:.2f}%",
        f"{metrics_rf['F1 (Macro)']*100:.2f}%"
    ],
    'Precision (Macro)': [
        f"{metrics_nn['Precision (Macro)']*100:.2f}%",
        f"{metrics_gb['Precision (Macro)']*100:.2f}%",
        f"{metrics_rf['Precision (Macro)']*100:.2f}%"
    ],
    'Recall (Macro)': [
        f"{metrics_nn['Recall (Macro)']*100:.2f}%",
        f"{metrics_gb['Recall (Macro)']*100:.2f}%",
        f"{metrics_rf['Recall (Macro)']*100:.2f}%"
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.set_index('Model', inplace=True)
display(summary_df)